In [ ]:
def clean_mta(turnstiles_df):
    """
    Converts input MTA data from total cumulative counts per 
    turnstile to new entries and exits at the time of each record. 
    Also adds datetime.
    
    
    """
    
    #convert column names to lower case and eliminate trailing spaces
    turnstiles_df.columns = map(lambda x: x.lower().replace(' ',''),
        turnstiles_df.columns)
    
    #sort by specific turnstiles and add converted counts
    df = turnstiles_df.sort_values(by = ['c/a','unit','scp','station',
        'linename','date','time'])
    df['coming'] = df['entries'].diff()
    df['going'] = df['exits'].diff()
    
    #add id to identify turnstiles with one column
    df['id'] = df['c/a']+df['unit']+df['scp']+df['station']+df['linename']
    
    #eliminate edges between different turnstiles
    mask = (df.id == df.id.shift(1)) & (df.id == df.id.shift(-1))
    df[['coming', 'going']] = df[['coming','going']].where(mask,
        other=np.nan)
    
    def rollover_correcter(value):
        #normal/possible values
        if 0 < value < 5000:
            return value
        #correct for turnstiles that are running backwards
        elif -5000 < value < 0:
            return -value
        #correct for turnstiles that have rolled over
        elif -2147483647 <= (value) < -2147478647:
            return value+2147483647
        #correct for backwards turnstiles that have rolled over
        elif 2147478647 < value <= 2147483647:
            return 2147483647-value
        #throw out unknown issues
        else:
            return np.nan
    
    df['entries'] = df.coming.apply(rollover_correcter)
    df['exits'] = df.going.apply(rollover_correcter)
    df.dropna(inplace=True)
    df['datetime'] = pd.to_datetime(df['date']+' '+df['time'])
    df.drop('coming', axis=1, inplace=True)
    df.drop('going', axis=1, inplace=True)
    df.drop('id', axis=1, inplace=True)
    return df